In [1]:
import os, sys
sys.path.insert(0, '..')

In [2]:
from fbpyutils import xlsx as XL, db as DB, ofx as OFX
import pandas as pd

In [3]:
OFX_PATH='/mnt/c/Users/fcjbispo/Meu Drive/Finanças/Extratos & Faturas/INTERPF_0001_19187904/OFX_DONE/Extrato-01-03-2023-a-31-03-2023.ofx'

In [4]:
ofx_data = OFX.read_from_path(OFX_PATH)

In [5]:
df = pd.DataFrame.from_dict(ofx_data['statement']['transactions'])

In [6]:
df.head()

,payee,type,date,amount,id,memo,sic,mcc,checknum
0,,PAYMENT,2023-03-31,-0.56,c6c508c607d2001776551e0ccb63c2a6,GLOBAL ACCOUNT,None,,077
1,,PAYMENT,2023-03-31,-51.24,f5c4427403b8efe4914ef15891de3581,GLOBAL ACCOUNT,None,,077
2,,PAYMENT,2023-03-31,-2.25,08065d552872b98a5ec709e57f53cbe6,GLOBAL ACCOUNT,None,,077
3,,PAYMENT,2023-03-31,-204.69,4ae9f7f3828fe09d15c819fcec5a48f9,GLOBAL ACCOUNT,None,,077
4,,PAYMENT,2023-03-31,-0.39,f58834a86ba10545c09fa7695bee91f2,CÂMBIO TRADE INTERNACIONAL,None,,077


In [7]:
xcols = ['type', 'date', 'amount']

In [8]:
df3 = DB.add_hash_column(df, 'xcol1', columns=xcols)

In [9]:
df3.head()

,xcol1,payee,type,date,amount,id,memo,sic,mcc,checknum
0,32b47b8838b2,,PAYMENT,2023-03-31,-0.56,c6c508c607d2001776551e0ccb63c2a6,GLOBAL ACCOUNT,None,,077
1,ecbe49753a5f,,PAYMENT,2023-03-31,-51.24,f5c4427403b8efe4914ef15891de3581,GLOBAL ACCOUNT,None,,077
2,c8717a66c116,,PAYMENT,2023-03-31,-2.25,08065d552872b98a5ec709e57f53cbe6,GLOBAL ACCOUNT,None,,077
3,0a6fbc1f3b5e,,PAYMENT,2023-03-31,-204.69,4ae9f7f3828fe09d15c819fcec5a48f9,GLOBAL ACCOUNT,None,,077
4,0fcf23368902,,PAYMENT,2023-03-31,-0.39,f58834a86ba10545c09fa7695bee91f2,CÂMBIO TRADE INTERNACIONAL,None,,077


In [10]:
df3 = DB.add_hash_column(df, 'xcol2')

In [11]:
df3.head()

,xcol2,payee,type,date,amount,id,memo,sic,mcc,checknum,xcol1
0,4767e17de387,,PAYMENT,2023-03-31,-0.56,c6c508c607d2001776551e0ccb63c2a6,GLOBAL ACCOUNT,None,,077,32b47b8838b2
1,11f4d290ff46,,PAYMENT,2023-03-31,-51.24,f5c4427403b8efe4914ef15891de3581,GLOBAL ACCOUNT,None,,077,ecbe49753a5f
2,dc0f9f4d33de,,PAYMENT,2023-03-31,-2.25,08065d552872b98a5ec709e57f53cbe6,GLOBAL ACCOUNT,None,,077,c8717a66c116
3,b10b70e6aeb3,,PAYMENT,2023-03-31,-204.69,4ae9f7f3828fe09d15c819fcec5a48f9,GLOBAL ACCOUNT,None,,077,0a6fbc1f3b5e
4,166f1516ce6b,,PAYMENT,2023-03-31,-0.39,f58834a86ba10545c09fa7695bee91f2,CÂMBIO TRADE INTERNACIONAL,None,,077,0fcf23368902


In [12]:
df3 = DB.add_hash_index(df3, 'id', columns=xcols)

In [14]:
df3.head(3)

,xcol2,payee,type,date,amount,id,memo,sic,mcc,checknum,xcol1
id,,,,,,,,,,,
32b47b8838b2,4767e17de387,,PAYMENT,2023-03-31,-0.56,c6c508c607d2001776551e0ccb63c2a6,GLOBAL ACCOUNT,None,,077,32b47b8838b2
ecbe49753a5f,11f4d290ff46,,PAYMENT,2023-03-31,-51.24,f5c4427403b8efe4914ef15891de3581,GLOBAL ACCOUNT,None,,077,ecbe49753a5f
c8717a66c116,dc0f9f4d33de,,PAYMENT,2023-03-31,-2.25,08065d552872b98a5ec709e57f53cbe6,GLOBAL ACCOUNT,None,,077,c8717a66c116


In [15]:
df4 = DB.add_hash_index(df3, 'id')

In [16]:
df4.head(3)

,xcol2,payee,type,date,amount,id,memo,sic,mcc,checknum,xcol1
id,,,,,,,,,,,
45ec7cd72f87,4767e17de387,,PAYMENT,2023-03-31,-0.56,c6c508c607d2001776551e0ccb63c2a6,GLOBAL ACCOUNT,None,,077,32b47b8838b2
1eb3d2a8dea7,11f4d290ff46,,PAYMENT,2023-03-31,-51.24,f5c4427403b8efe4914ef15891de3581,GLOBAL ACCOUNT,None,,077,ecbe49753a5f
b9e84c2b2b02,dc0f9f4d33de,,PAYMENT,2023-03-31,-2.25,08065d552872b98a5ec709e57f53cbe6,GLOBAL ACCOUNT,None,,077,c8717a66c116
